# Homework 3

In this notebook we will present our solution for the third homework of ADA.

In [2]:
#Importing all the libraries
import pandas as pd
import numpy as np
import folium as fl
from lxml import etree

We begin by importing the whole CSV thanks to the pandas library.

In [2]:
path = 'Data/P3_GrantExport.csv'

#Reading the csv with Pandas

data = pd.read_csv(path,sep = ';')

We first take a look at the data!

In [3]:
data.head(100)

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN
5,8,Die schweizerische Wirtschaftspolitik seit dem...,NaN,Kleinewefers Henner,Project funding (Div. I-III),Project funding,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,10203,Economics,"Human and Social Sciences;Economics, law",01.01.1976,31.12.1978,53009.00,NaN
6,9,Theologische Forschungen zur Oekumene (Studien...,NaN,Stirnimann Heinrich,Project funding (Div. I-III),Project funding,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,10102,"Religious sciences, Theology",Human and Social Sciences;Theology & religious...,01.01.1976,31.12.1976,25403.00,NaN
7,10,Konfuzianische Kulturwerte in der sozialen Ent...,NaN,Deuchler Martina,Project funding (Div. I-III),Project funding,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,10301,History in general,Human and Social Sciences;Theology & religious...,01.10.1975,31.03.1977,47100.00,NaN
8,11,Edizione degli scritti di Aurelio de' Giorgi B...,NaN,Stäuble Antonio,Project funding (Div. I-III),Project funding,NaN,Université de Lausanne - LA,10502,Romance languages and literature,Human and Social Sciences;Linguistics and lite...,01.10.1975,31.03.1977,25814.00,NaN
9,13,La construction de nouveautés au sein des morp...,NaN,Piaget Jean,Project funding (Div. I-III),Project funding,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,10105,Psychology,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1978,360000.00,NaN


We first begin by looking at the "Approved Amount" filed, which contains the amount the is given to the university.

In [4]:
data['Approved Amount'].value_counts().head()

data not included in P3    10910
10000.00                     611
5000.00                      348
8000.00                      320
15000.00                     273
Name: Approved Amount, dtype: int64

We can see that for 1/6 of the funding there is no amount that is registered, therefore we can remove them from the database as we will not use them for the sum of grant money in each canton.

In [5]:
data_clean = data[data['Approved Amount'].isin(['data not included in P3']) == False]

By reading the [SNF](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) page we also find that if the research is not carried out at a swiss institution, the 'University' field will be left blank. As we are not interested in funding that goes outside of Switzerland we can also delete these rows.

In [6]:
swissFunding = data_clean.dropna(subset=['University'])

As we can see that made us drop a large part of the data already.

In [7]:
print('Initial size of data : ', len(data))
print('Usable data for switzerland : ',len(swissFunding))
print('Percentage of Data kept',round(len(swissFunding)/len(data),3)*100,' %')

Initial size of data :  63969
Usable data for switzerland :  50878
Percentage of Data kept 79.5  %


For this analysis, we are only interested in the University name and the funding amount so that we can sort by cantons, we will therefore drop the other columns. We also change the type of the approved amount to float so that we can use them as numbers afterwards.

In [8]:
swissFunding = swissFunding[['University','Approved Amount']]
swissFunding['Approved Amount'] = swissFunding['Approved Amount'].astype(np.float)
swissFunding.dtypes

University          object
Approved Amount    float64
dtype: object

We then proceed to group by university.

In [9]:
uniGroup = swissFunding.groupby('University')

We then use the aggregate sum function to get the total funding per university.

In [10]:
fundingUni = uniGroup.sum()
fundingUni.head()

,Approved Amount
University,
AO Research Institute - AORI,3435621.0
Allergie- und Asthmaforschung - SIAF,19169965.0
Berner Fachhochschule - BFH,31028695.0
Biotechnologie Institut Thurgau - BITG,2492535.0
Centre de rech. sur l'environnement alpin - CREALP,1567678.0


In [11]:
fundingUni['Canton'] = ''
fundingUni.head()

,Approved Amount,Canton
University,,
AO Research Institute - AORI,3435621.0,
Allergie- und Asthmaforschung - SIAF,19169965.0,
Berner Fachhochschule - BFH,31028695.0,
Biotechnologie Institut Thurgau - BITG,2492535.0,
Centre de rech. sur l'environnement alpin - CREALP,1567678.0,


Definition of a function that will query the geonames API and return the name of a Canton if it appears in the search.


In [9]:
def query_term(term):
    
    if term != '-' and term != 'de' and term != 'di' and term != 'und' and term != 'of':
        

        term_value = term

        tree = etree.parse("http://api.geonames.org/search?name=%s&maxRows=200&country=CH&username=axeldlh" %  (term_value))
        for user in tree.xpath("/geonames/geoname/fcode"):
            #print(user.text)
            if user.text == "ADM1":

                for i in user.itersiblings(preceding=True):
                    if i.tag == 'name':
                        return i.text
                    
        return 'no canton found'
    else:
        return 'no canton found'

print(query_term('Swiss Institute of Bioinformatics - SIB'))

no canton found


In [32]:
def query_city(term):
    lat_lng = []
    if term != '-' and term != 'de' and term != 'di' and term != 'und' and term != 'of' and term != '':
        

        term_value = term

        tree = etree.parse("http://api.geonames.org/search?name_equals=%s&maxRows=100&country=CH&username=axeldlh" %  (term_value))
        for user in tree.xpath("/geonames/geoname/fcode"):
            #print(user.text)
            if user.text == "PPL" or user.text =="PPLA":

                for i in user.itersiblings(preceding=True):
                    if i.tag == 'lng':
                        lat_lng.append(i.text)
                    if i.tag == 'lat':
                        lat_lng.append(i.text)
                        
                        
                return lat_lng
                        
                    
        return 'no city found'
    else:
        return 'no city found'
    
a = query_city('')
print(a)
print(type(a[1]))

no city found
<class 'str'>


In [33]:
def canton_geolocalization(lat, lng):
    
    lat_value = lat
    lng_value = lng
    
    tree = etree.parse("http://api.geonames.org/extendedFindNearby?lat=%s&lng=%s&username=axeldlh" %  (lat_value, lng_value))
    for user in tree.xpath("/geonames/geoname/fcode"):
            #print(user.text)
            if user.text == "ADM1":

                for i in user.itersiblings(preceding=True):
                    if i.tag == 'name':
                        return i.text
                   
    return 'no canton found'


b = canton_geolocalization(a[1], a[0])
print(b)

no canton found


In [34]:
http://api.geonames.org/findNearby?lat=46.516&lng=6.63282&radius=50&featureClass=A&featureCode=ADM1&username=axeldlh
    
    http://api.geonames.org/extendedfindNearby?lat=%s&lng=%s&radius=30&featureClass=A&featureCode=ADM1&username=axeldlh

SyntaxError: invalid syntax (<ipython-input-34-1004d6d84db2>, line 1)

In [35]:
print(fundingUni.index.shape)

for i in range(fundingUni.index.shape[0]):
    
    for term in fundingUni.index[i].split():
        print(term)



(77,)
AO
Research
Institute
-
AORI
Allergie-
und
Asthmaforschung
-
SIAF
Berner
Fachhochschule
-
BFH
Biotechnologie
Institut
Thurgau
-
BITG
Centre
de
rech.
sur
l'environnement
alpin
-
CREALP
EPF
Lausanne
-
EPFL
ETH
Zürich
-
ETHZ
Eidg.
Anstalt
für
Wasserversorgung
-
EAWAG
Eidg.
Forschungsanstalt
für
Wald,Schnee,Land
-
WSL
Eidg.
Hochschulinstitut
für
Berufsbildung
-
EHB
Eidg.
Material
und
Prüfungsanstalt
-
EMPA
Ente
Ospedaliero
Cantonale
-
EOC
Fachhochschule
Kalaidos
-
FHKD
Fachhochschule
Nordwestschweiz
(ohne
PH)
-
FHNW
Fachhochschule
Ostschweiz
-
FHO
Facoltà
di
Teologia
di
Lugano
-
FTL
Fernfachhochschule
Schweiz
(Mitglied
SUPSI)
-
FFHS
Firmen/Privatwirtschaft
-
FP
Forschungsanstalten
Agroscope
-
AGS
Forschungsinstitut
für
Opthalmologie
-
IRO
Forschungsinstitut
für
biologischen
Landbau
-
FIBL
Forschungskommission
SAGW
Franklin
University
Switzerland
-
FUS
Friedrich
Miescher
Institute
-
FMI
HES
de
Suisse
occidentale
-
HES-SO
Haute
école
pédagogique
BE,
JU,
NE
-
HEPBEJUNE
Haute
école
pédag

In [36]:
print(fundingUni.index.shape)

for i in range(fundingUni.index.shape[0]):
    
    canton_name = 'not found'
    
    for term in fundingUni.index[i].split():
        result = query_term(term)
        if result != 'no canton found':
            canton_name = result
    
    if canton_name == 'not found':
        for term_2 in fundingUni.index[i].split():
            coor = query_city(term_2)
            
            if coor != 'no city found':
                canton_name = canton_geolocalization(coor[1], coor[0])
                

    fundingUni['Canton'][i] = canton_name
    
fundingUni.head()

(77,)


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Approved Amount,Canton
University,,
AO Research Institute - AORI,3435621.0,not found
Allergie- und Asthmaforschung - SIAF,19169965.0,not found
Berner Fachhochschule - BFH,31028695.0,not found
Biotechnologie Institut Thurgau - BITG,2492535.0,Thurgau
Centre de rech. sur l'environnement alpin - CREALP,1567678.0,Canton Grisons


In [38]:
fundingUni.head(100)
count_not_found = fundingUni['Canton'][fundingUni['Canton'] == 'not found'].count()

print(count_not_found/fundingUni['Canton'].count()*100)
fundingUni.head(77)


46.7532467532


,Approved Amount,Canton
University,,
AO Research Institute - AORI,3.435621e+06,not found
Allergie- und Asthmaforschung - SIAF,1.916996e+07,not found
Berner Fachhochschule - BFH,3.102870e+07,not found
Biotechnologie Institut Thurgau - BITG,2.492535e+06,Thurgau
Centre de rech. sur l'environnement alpin - CREALP,1.567678e+06,Canton Grisons
EPF Lausanne - EPFL,1.175316e+09,Vaud
ETH Zürich - ETHZ,1.635597e+09,Zurich
Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,not found
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",4.836039e+07,not found


In [10]:
#In order to map the university name to the canton, we need to use the Geonames API. This API allows us to get xml data
#containing multiple location information related to the specific search term. By crawling in the xml code using etree,
#we are able to retrieve all these informations. We specifically look for the fcode ADM1 which represent the canton 
#in switzerland (See: http://www.geonames.org/export/codes.html). When we found the right Geonames tag containing an 
#ADM1 fcode, we simply crawl into his tag sibling using itersiblings and put the information into a dictionary that
#is subsequently transformed into a DataFrame

from lxml import etree

canton_dic = {}

value = 'Zurich'

tree = etree.parse("http://api.geonames.org/search?q=%s&maxRows=100&country=CH&username=axeldlh" %  (value))
for user in tree.xpath("/geonames/geoname/fcode"):
    #print(user.text)
    if user.text == "ADM1":
        print('hello')
        canton_dic[user.tag] = user.text
        for i in user.itersiblings(preceding=True):
            canton_dic[i.tag] = i.text
            print(i.text)
            print(i.tag)


            
            

canton_dataframe = pd.DataFrame(canton_dic, index=[0])
canton_dataframe.head()

hello
A
fcl
Switzerland
countryName
CH
countryCode
2657895
geonameId
8.66667
lng
47.41667
lat
Zurich
name
Kanton Zürich
toponymName


,countryCode,countryName,fcl,fcode,geonameId,lat,lng,name,toponymName
0,CH,Switzerland,A,ADM1,2657895,47.41667,8.66667,Zurich,Kanton Zürich


In [ ]:
%%capture
#This allows choropleth to load the maps and map to the values in the data frames. The key_on Feature
#feature_id should be created in the dataframe if it does not exist. 
# In the DataFrame all id present in the topoJSON must be present in the column of DataFrame.
state_geo = r'map.json'

state_data = pd.read_csv(state_unemployment)
map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(geo_path=cantons_geo, 
                     data=df,
                     columns=['Cantons', 'Values'],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='YlGn',
                     legend_name = 'Random numbers'
                    )